# Importing necessary packages

In [38]:
import pandas as pd  # For handling the data

import seaborn as sns  # For plotting
import matplotlib.pyplot as plt  # for subplots
import plotly_express as px  # for interactiv plots

# import warnings
# warnings.filterwarnings(action="ignore")

import plotly.io as pio

pio.templates.default = "gridon"  # Sets "gridon" as default for all Plotly graphs


# Task 2
Warm up Vaccine Data
___

In [39]:
# read in data:
df_vaccine = pd.read_excel(
    "../Lab_Data/Vaccine.xlsx", sheet_name="Vaccinerade kommun och ålder"
)


In [40]:
# Initial information gathering.

# df_vaccine.info()  # Län_namn, Kommun_namn and Ålder are objects. rest are int or floats
df_vaccine.isnull().sum()  # Ammount of nulls are: Antal 3 doser 580, Antal 4 doser 2030, Andel 3 doser 580, Andel 4 doser 2030


Län                       0
Län_namn                  0
Kommun                    0
Kommun_namn               0
Ålder                     0
Befolkning                0
Antal minst 1 dos         0
Antal minst 2 doser       0
Antal 3 doser           580
Antal 4 doser          2030
Andel minst 1 dos         0
Andel minst 2 doser       0
Andel 3 doser           580
Andel 4 doser          2030
dtype: int64

In [41]:
df_vaccine.rename(
    columns={"Ålder": "Age group"}, inplace=True
)  # Renaming Ålder into Age group.
df_vaccine.replace(
    {"90 eller äldre": "90 or older"},
    inplace=True,
)


a) How many regional Regional Councils are represented in the data set.

In [42]:
len(df_vaccine["Län"].drop_duplicates().value_counts())
# answer = 21


21

b) How many municipalitys are represented in the data set?

In [43]:
len(df_vaccine["Kommun"].drop_duplicates().value_counts())
# Answer is 290 Kommuner.


290

c) How large is the population in the data set?

In [44]:
sheet_population = df_vaccine["Befolkning"].sum()
# Answer is 9 092 790
print(f"Population according to sheet = {sheet_population}")


Population according to sheet = 9092790


d) Calculate how many children that are under 18 years old in Sweden based on this data sheet.

In [45]:
# Håkan helped me with this code, my previous attempt was a for loop. Changed into this instead due to efficency.
sheet_population_under_18 = df_vaccine[
    (df_vaccine["Age group"] == "12-15") | (df_vaccine["Age group"] == "16-17")
]["Befolkning"].sum()
print(
    f"Amount of children under the age of 18 represented in sheet = {sheet_population_under_18}"
)


Amount of children under the age of 18 represented in sheet = 745370


d)2 Calculate how many children under the age of 18 when compared to actual data for population in sweden.

In [54]:
# Calculating how many children are below 12:
swedish_population = 10452326  # Source: https://www.scb.se/hitta-statistik/sverige-i-siffror/manniskorna-i-sverige/sveriges-befolkning/
age_0_11 = (
    swedish_population - df_vaccine["Befolkning"].sum()
)  # Calculates value for how many children between 0-11
print(
    f"Amount of children in age group 0-11: {age_0_11}. Total population: {sheet_population + age_0_11}"
)

swedish_population_all = df_vaccine.append(
    {"Age group": "0-11", "Befolkning": age_0_11}, ignore_index=True
)
# Creating a new variable with all agegroups
swedish_population_all.sort_values(
    by="Age group", ascending=True, inplace=True
)  # Sorts them so 0-11 is displayed first


Amount of children in age group 0-11: 1359536. Total population: 10452326


In [56]:
swedish_population_under_18 = swedish_population_all[
    (swedish_population_all["Age group"] == "0-11")
    | (swedish_population_all["Age group"] == "12-15")
    | (swedish_population_all["Age group"] == "16-17")
]["Befolkning"].sum()
print(f"Real amount of children under the age of 18 = {swedish_population_under_18}")


Real amount of children under the age of 18 = 2104906
